In [1]:
""" Rasterize Aqueduct 21 and store to Google Cloud Storage.
-------------------------------------------------------------------------------

Author: Rutger Hofste
Date: 20190522
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

"""

TESTING = 0

SCRIPT_NAME = "Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01"
OUTPUT_VERSION = 2


GDAL_RASTERIZE_PATH = "/opt/anaconda3/envs/python35/bin/gdal_rasterize"
X_DIMENSION_30S = 43200
Y_DIMENSION_30S = 21600

S3_TABLE_INPUT_PATH = "s3://wri-projects/Aqueduct2x/Aqueduct21Data/aqueduct_global_maps_21_shp"
TABLE_INPUT_FILE_NAME = "aqueduct_global_dl_20150409.shp"

ec2_input_path = "/volumes/data/{}/input_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION) 
ec2_output_path =  "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION) 

GCS_OUTPUT_PATH = "gs://aqueduct30_v01/{}/".format(SCRIPT_NAME)


INDICATORS = {"owr_score":"DEFAULT",
              "bws_score":"BWS_s",
              "iav_score":"WSV_s",
              "sev_score":"SV_s"}



print("\nGCS_OUTPUT_PATH: " + GCS_OUTPUT_PATH)


GCS_OUTPUT_PATH: gs://aqueduct30_v01/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

%matplotlib inline

Y2019M05D22 UTC 18:02


In [3]:
import os
import subprocess
import numpy as np
import pandas as pd
import geopandas as gpd

In [4]:
!rm -r {ec2_input_path}
!rm -r {ec2_output_path}
!mkdir -p {ec2_input_path}
!mkdir -p {ec2_output_path}

rm: cannot remove '/volumes/data/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/input_V02': No such file or directory
rm: cannot remove '/volumes/data/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/output_V02': No such file or directory


In [5]:
!aws s3 cp {S3_TABLE_INPUT_PATH} {ec2_input_path} --recursive

download: s3://wri-projects/Aqueduct2x/Aqueduct21Data/aqueduct_global_maps_21_shp/aqueduct_global_dl_20150409.cpg to ../../../../../data/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/input_V02/aqueduct_global_dl_20150409.cpg
download: s3://wri-projects/Aqueduct2x/Aqueduct21Data/aqueduct_global_maps_21_shp/aqueduct_global_dl_20150409.prj to ../../../../../data/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/input_V02/aqueduct_global_dl_20150409.prj
download: s3://wri-projects/Aqueduct2x/Aqueduct21Data/aqueduct_global_maps_21_shp/dictionary_20150409.xlsx to ../../../../../data/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/input_V02/dictionary_20150409.xlsx
download: s3://wri-projects/Aqueduct2x/Aqueduct21Data/aqueduct_global_maps_21_shp/aqueduct_global_dl_20150409.shp.xml to ../../../../../data/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/input_V02/aqueduct_global_dl_20150409.shp.xml
download: s3://wri-projects/Aqueduct2x/Aqueduct21Data/aqueduct_global_maps_21_shp/aqueduct_global_maps_21.

In [6]:
table_input_path = "{}/{}".format(ec2_input_path,TABLE_INPUT_FILE_NAME)

In [7]:
gdf_geom = gpd.read_file(table_input_path)

In [8]:
gdf_geom.head()

,GU,Shape_Leng,Shape_Area,BasinID,COUNTRY,BASIN_NAME,WITHDRAWAL,CONSUMPTIO,BA,BWS,...,W_CHEM,W_POWER,W_MINE,W_OILGAS,DEF_PQUANT,W_AGR,W_FOODBV,W_TEX,OWR_cat,geometry
0,1,0.559986,0.005929,439,Guinea Bissau,,318530368.0,2.191088e+08,9.921439e+09,0.032105,...,2.882769,1.652696,2.817264,3.574223,0.953074,1.674275,2.102334,1.782311,Low to medium risk (1-2),(POLYGON ((-14.75229565619014 12.6188368125272...
1,2,2.272372,0.099605,441,Guinea Bissau,,14757104.0,5.604148e+06,5.116468e+09,0.002884,...,2.841154,1.564447,2.664640,3.529458,0.826442,1.600816,2.142925,1.746387,Low to medium risk (1-2),(POLYGON ((-16.13767726797875 12.2940957333858...
2,3,0.610379,0.017086,444,Guinea,GEBA,34460596.0,1.764853e+07,9.535255e+09,0.003614,...,2.774803,1.551692,2.707415,3.352587,0.891978,1.582219,2.270701,1.773006,Low to medium risk (1-2),POLYGON ((-13.56238156611261 12.66742547396541...
3,4,5.384966,0.636739,444,Guinea Bissau,GEBA,34460596.0,1.764853e+07,9.535255e+09,0.003614,...,2.897160,1.571053,2.834815,3.544556,0.891978,1.660348,2.335788,1.858815,Medium to high risk (2-3),POLYGON ((-13.73217699483075 12.57834249376896...
4,5,1.815190,0.055072,446,Guinea Bissau,,6855107.0,2.852844e+06,1.839854e+09,0.003726,...,2.878972,1.549313,2.687177,3.524936,0.819780,1.629414,2.316616,1.815062,Low to medium risk (1-2),(POLYGON ((-15.72857010960513 11.9712600703465...


In [9]:
for indicator_aq30, indicator_aq21 in INDICATORS.items():
    print(indicator_aq30, indicator_aq21)
    
    column = indicator_aq21
    layer = "aqueduct_global_dl_20150409"
    destination_path_shp = ec2_input_path
    destination_path_tif = "{}/{}.tif".format(ec2_output_path,indicator_aq21)
    command = "{} -a {} -at -ot Float32 -of GTiff -te -180 -90 180 90 -ts {} {} -co COMPRESS=DEFLATE -co PREDICTOR=1 -co ZLEVEL=6 -l {} -a_nodata -32767 {} {}".format(GDAL_RASTERIZE_PATH,column,X_DIMENSION_30S,Y_DIMENSION_30S,layer,destination_path_shp,destination_path_tif)
    print(command)
    response = subprocess.check_output(command,shell=True)

iav_score WSV_s
/opt/anaconda3/envs/python35/bin/gdal_rasterize -a WSV_s -at -ot Float32 -of GTiff -te -180 -90 180 90 -ts 43200 21600 -co COMPRESS=DEFLATE -co PREDICTOR=1 -co ZLEVEL=6 -l aqueduct_global_dl_20150409 -a_nodata -32767 /volumes/data/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/input_V02 /volumes/data/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/output_V02/WSV_s.tif
sev_score SV_s
/opt/anaconda3/envs/python35/bin/gdal_rasterize -a SV_s -at -ot Float32 -of GTiff -te -180 -90 180 90 -ts 43200 21600 -co COMPRESS=DEFLATE -co PREDICTOR=1 -co ZLEVEL=6 -l aqueduct_global_dl_20150409 -a_nodata -32767 /volumes/data/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/input_V02 /volumes/data/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/output_V02/SV_s.tif
owr_score DEFAULT
/opt/anaconda3/envs/python35/bin/gdal_rasterize -a DEFAULT -at -ot Float32 -of GTiff -te -180 -90 180 90 -ts 43200 21600 -co COMPRESS=DEFLATE -co PREDICTOR=1 -co ZLEVEL=6 -l aqueduct_global_dl_20150409 -a_nodata -3276

In [10]:
!gsutil -m cp -r {ec2_output_path} {GCS_OUTPUT_PATH}

Copying file:///volumes/data/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/output_V02/WSV_s.tif [Content-Type=image/tiff]...
Copying file:///volumes/data/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/output_V02/BWS_s.tif [Content-Type=image/tiff]...
Copying file:///volumes/data/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/output_V02/DEFAULT.tif [Content-Type=image/tiff]...
Copying file:///volumes/data/Y2019M05D22_RH_AQ39VS21_Rasterize_AQ21_EE_V01/output_V02/SV_s.tif [Content-Type=image/tiff]...
- [4/4 files][ 59.5 MiB/ 59.5 MiB] 100% Done                                    
Operation completed over 4 objects/59.5 MiB.                                     


In [11]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:07:58.563173
